In [1]:
import os
import pandas as pd
import re
import numpy as np

import collections
from operator import itemgetter
import pprint as pp
import re
from nltk.corpus import stopwords
import nltk.data


#data from http://times.cs.uiuc.edu/~wang296/Data/
files = os.listdir('./Review_Texts')
#print(os.listdir('./Review_Texts'))
print(len(files))

1759


In [2]:
import time

In [3]:
cats = ['Rooms', 'Date', 'Location', 'Service', 'Business service', 'Author', 'Check in / front desk', 'No. Helpful', 'Cleanliness', 'Content', 'Value', 'No. Reader', 'Overall']
data = pd.DataFrame(columns=cats)

In [4]:
def addFileToData(filename, data):
    intColumns = ['No. Reader', 'No. Helpful', 'Cleanliness','Check in / front desk', 'Value', 'Overall', 'Service', 'Business service', 'Rooms', 'Location']
    
    with open(filename, 'r') as content_file:
        content = content_file.read()

    #print(content)
    reviews = content.split("\n\n")
    
    for r in reviews:
        #print(r)
        thisReview = pd.Series([None]*len(cats), cats)
        splt = r.split("\n")
        for s in splt:
            for c in cats:
                if "<"+c+">" in s:
                    value = s.replace('<'+c+'>', '')
                    if c in intColumns:
                        value = int(value)
                    if value == -1: #we dont want -1 as this is going to mess up averaging, take np.nan
                        value = np.nan

                    if c == "Content":
                        value = value.lower()

                    thisReview[c] = value
        #print(thisReview)
        if not thisReview["Content"] == None and len(thisReview["Content"]) > 60:
            data = data.append(thisReview, ignore_index=True)
    return data
        
data = addFileToData('./Review_Texts/hotel_100504.dat', data) 
data = addFileToData('./Review_Texts/hotel_72572.dat', data) 


In [5]:
print(data.shape)
data.head()

(199, 13)


,Rooms,Date,Location,Service,Business service,Author,Check in / front desk,No. Helpful,Cleanliness,Content,Value,No. Reader,Overall
0,5.0,"Dec 23, 2008",5.0,5.0,NaN,selizabethm,5.0,NaN,5.0,wonderful time- even with the snow! what a gre...,4.0,NaN,5.0
1,4.0,"Nov 13, 2008",5.0,5.0,NaN,IndieLady,5.0,NaN,4.0,"lovely hotel, unique decor, friendly front des...",5.0,NaN,4.0
2,4.0,"Nov 11, 2008",3.0,NaN,4,Hilobb,5.0,NaN,4.0,"nice hotel, expensive parking we got a good de...",4.0,NaN,4.0
3,5.0,"Nov 4, 2008",5.0,5.0,5,Chianti_girl24,5.0,NaN,5.0,fabulous hotel location and service are great....,5.0,NaN,5.0
4,NaN,"Oct 18, 2008",NaN,NaN,NaN,hothearted,NaN,2,NaN,"loved the monaco! staff was amazing, with a sm...",NaN,2,5.0


In [6]:
#mean of int columns
data.mean(axis=0)

Rooms                    4.208589
Location                 4.296875
Service                  4.167702
Check in / front desk    4.273438
Cleanliness              4.518293
Value                    3.969136
Overall                  4.165829
dtype: float64

In [ ]:
data["Content"][1]

In [8]:
#creating the vocab of all the words

def lintWord(w):
    regex = re.compile(r'(,|\(|\)|!|:|$|\.)')
    w = re.sub(regex, '', w)
    return w


allWords = ""
for r in data["Content"]:
    #add word to big content string
    allWords += r + " "
    
#split the string at spaces, keep only unique
words = set(allWords.split(" "))


vocab = list(set([lintWord(w) for w in words if not w in stopwords.words("english")]))

#n eed to remove stopwords again because some of them may have had punctuation 
# at the end and didnt get caught the first time
vocab = [w for w in vocab if not w in stopwords.words("english") and len(w) > 2]

#vocab


In [9]:
print(len(vocab))
print(len(set(vocab)))
print("i" in vocab)


3776
3776
False


In [10]:

"""Algorithm: Aspect Segmentation Algorithm
Input: A collection of reviews {푑1, 푑2, . . . , 푑∣퐷∣)}, set of
aspect keywords {푇1, 푇2, . . . , 푇푘}, vocabulary V, selection
threshold p and iteration step limit I.
Output: Reviews split into sentences with aspect assignments.
Step 0: Split all reviews into sentences, 푋 =
{푥1, 푥2, . . . , 푥푀};
Step 1: Match the aspect keywords in each sentence
of X and record the matching hits for each aspect i in
퐶표푢푛푡(푖);
Step 2: Assign the sentence an aspect label by 푎푖 =
푎푟푔푚푎푥푖 퐶표푢푛푡(푖). If there is a tie, assign the sentence
with multiple aspects.
Step 3: Calculate chi^2 measure of each word (in V);
Step 4: Rank the words under each aspect with respect
to their chi^2value and join the top p words for each aspect
into their corresponding aspect keyword list 푇푖;
Step 5: If the aspect keyword list is unchanged or iteration
exceeds I, go to Step 6, else go to Step 1;
Step 6: Output the annotated sentences with aspect
assignments."""

'Algorithm: Aspect Segmentation Algorithm\nInput: A collection of reviews {푑1, 푑2, . . . , 푑∣퐷∣)}, set of\naspect keywords {푇1, 푇2, . . . , 푇푘}, vocabulary V, selection\nthreshold p and iteration step limit I.\nOutput: Reviews split into sentences with aspect assignments.\nStep 0: Split all reviews into sentences, 푋 =\n{푥1, 푥2, . . . , 푥푀};\nStep 1: Match the aspect keywords in each sentence\nof X and record the matching hits for each aspect i in\n퐶표푢푛푡(푖);\nStep 2: Assign the sentence an aspect label by 푎푖 =\n푎푟푔푚푎푥푖 퐶표푢푛푡(푖). If there is a tie, assign the sentence\nwith multiple aspects.\nStep 3: Calculate chi^2 measure of each word (in V);\nStep 4: Rank the words under each aspect with respect\nto their chi^2value and join the top p words for each aspect\ninto their corresponding aspect keyword list 푇푖;\nStep 5: If the aspect keyword list is unchanged or iteration\nexceeds I, go to Step 6, else go to Step 1;\nStep 6: Output the annotated sentences with aspect\nassignments.'

In [11]:
#tokenizer to split sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


In [14]:
# initial seeds from LARA paper
seeds = {"Value" : ["value", "price", "quality","worth"],
         "Rooms" : ["room", "suite", "view", "bed"],
         "Location" : ["location", "traffic", "minute", "restaurant"],
         "Cleanliness" : ["clean", "dirty", "maintain", "smell"],
         "Check in / front desk": ["stuff", "check", "help", "reservation"],
         "Service" : ["service", "food", "breakfast", "buffet"],
         "Business service" : ["business", "center", "computer", "internet"]
        }


def aspectSegmentation(reviews, aspects, vocab=[], threshold=0, iterationLimit=3):
    #when we have the top chi-squared rated keywords, how many do we take
    keywordsToTake = 3
    
    #bootstrap iterations
    for i in range(0, iterationLimit):
        
        #print our current aspects
        print("begin bootstrapping iteration, aspect keywords: ")
        pp.pprint(aspects)
        print("\n\n\n")
        labeledSentences = []
        for r in reviews:
            #use the pickle tokenizer to split sentences
            sentences = tokenizer.tokenize(r)
            
            for s in sentences:
                
                theseAspects = collections.defaultdict(int)
                maxAspect = (0, "None")
                
                #for each aspect count how many times one of those aspect words appears
                for a in aspects:
                    for word in aspects[a]:
                        if " "+word+" " in s:
                            theseAspects[a] += 1
                
                #find the max occuring aspect for each sentence, take multiple if ties
                for a in theseAspects:
                    if theseAspects[a] > maxAspect[0]:
                        maxAspect = (theseAspects[a], a)
                    if theseAspects[a] == maxAspect[0] and a not in maxAspect:
                        #label it with multiple aspects
                        maxAspect = maxAspect + (a, )
                
                #add the sentence with labels
                labeledSentences.append((s, maxAspect[1:]))
            
        
        chiSquaredForAspects = collections.defaultdict(list)
        #calculate chi squared measure for each word in vocab
        """c1 is the number of times w occurs in sentences belonging
            to aspect a_i, c2 is the number of times w occurs
            in sentences not belonging to a_i, c3 is the number of sentences
            of aspect a_i that do not contain w, c4 is the number
            of sentences that neither belong to aspect a_i, nor contain
            word w, and C is the total number of word occurrences"""
        for w in vocab:
            for a in aspects:
                c = 0
                c_1 = 0
                c_2 = 0 
                c_3 = 0
                c_4 = 0

                for s in labeledSentences:
                    sentenceText = s[0]
                    sentenceAspects = s[1]

                    if " "+w+" " in sentenceText and a in sentenceAspects:
                        c_1 += 1
                    elif " "+w+" " in sentenceText and a not in sentenceAspects:
                        c_2 += 1
                    elif a in sentenceAspects and w not in sentenceText:
                        c_3 += 1
                    else:
                        c_4 += 1

                numer = ((1.0 * c_1 * c_4 - 1.0 * c_2 * c_3)**2)
                denom = (1.0*(c_1 + c_3) * 1.0 * (c_2 + c_4) * 1.0 * (c_1 + c_2) * 1.0 * (c_3 + c_4))
                #unreasonable use of 1.0's here to be safe
                if denom != 0:
                    csq = numer / denom
                    chiSquaredForAspects[a].append((w, csq))


        #have the chi squared aspects for each word in vocab, add top kewordsToTake for each aspect
        for a in chiSquaredForAspects:
            #make sure were not taking words we already have
            noDupes = [tup for tup in chiSquaredForAspects[a] if tup[0] not in aspects[a]]
            chiSquaredForAspects[a] = sorted(noDupes, key=itemgetter(1), reverse=True)[0:keywordsToTake]
            for t in chiSquaredForAspects[a]:
                if t[0] not in aspects[a] and t[0] != '':
                    aspects[a].append(t[0])
            
            
        
            #split into sentences
        
        #loop through again
    return labeledSentences
    #return labeledSentences

start = time.time()
sentencesWLabels = aspectSegmentation(data["Content"], seeds, vocab)
end = time.time()

print("done, time taken:", end-start)

begin bootstrapping iteration, aspect keywords: 
{'Business service': ['business', 'center', 'computer', 'internet'],
 'Check in / front desk': ['stuff', 'check', 'help', 'reservation'],
 'Cleanliness': ['clean', 'dirty', 'maintain', 'smell'],
 'Location': ['location', 'traffic', 'minute', 'restaurant'],
 'Rooms': ['room', 'suite', 'view', 'bed'],
 'Service': ['service', 'food', 'breakfast', 'buffet'],
 'Value': ['value', 'price', 'quality', 'worth']}




begin bootstrapping iteration, aspect keywords: 
{'Business service': ['business',
                      'center',
                      'computer',
                      'internet',
                      'wireless',
                      'access',
                      'fitness'],
 'Check in / front desk': ['stuff',
                           'check',
                           'help',
                           'reservation',
                           'persons',
                           'skills',
                           'you鈥檒

In [17]:
for s in sentencesWLabels:
    print(s)
    print("\n")

('wonderful time- even with the snow!', ('None',))


('what a great experience!', ('Location',))


('from the goldfish in the room (which my daughter loved) to the fact that the valet parking staff who put on my chains on for me it was fabulous.', ('Rooms',))


('the staff was attentive and went above and beyond to make our stay enjoyable.', ('None',))


("oh, and about the parking: the charge is about what you would pay at any garage or lot- and i bet they wouldn't help you out in the snow! ", ('Check in / front desk',))


("lovely hotel, unique decor, friendly front desk staff, central location it's only recently that i've started staying at boutique hotels, so i'm definitely not an expert on the subtleties that these types of establishments provide.", ('Location',))


("but i've already stayed at a few kimptons and i must say that my experiences are mostly good.", ('None',))


('hotel monaco is no exception.', ('None',))


('the decor at the kimpton hotels is always unique - they ha

In [ ]:
#seeds will have changed, whats differet
seeds

In [ ]:
def seedInReview(review, seeds):
    for s in seeds:
        #print(s)
        if s in review:
            return True
    return False


reviewsWithSeeds = {}
for s in seeds:
    reviewsWithSeeds[s] = sum(data["Content"].apply(seedInReview, args = [seeds[s]]))
reviewsWithSeeds

In [ ]:



# from gensim.models import word2vec
# print("training")
# model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features)
